## Transportation & distribution problems solved by PuLP

Objective: supply and demand optimization on all necessary routes while minimizing total cost

<img src="https://www.coin-or.org/PuLP/_images/brewery_arcs.jpg" width="500"/>

Costs to transport products from Warehouse to Houses

From Warehouse to House	| A	| B
-|-|-
1	|2	|3
2	|4	|1
3	|5	|3
4	|2	|2
5	|1	|3

In [1]:
import pandas as pd
import numpy as np

import pulp

import warnings
warnings.filterwarnings('ignore')

In [62]:
warehouses = ['A', 'B']
houses = ['1', '2', '3', '4', '5']

supply = {'A' : 1000, 'B' : 4000}
demand = {'1' : 500, '2' : 900, '3' : 1800, '4' : 200, '5' : 700}

costs = [[2, 4, 5, 2, 1], 
         [3, 1, 3, 2, 3]]

costs = pulp.makeDict([warehouses, houses], costs, None)

Create optimization object

In [81]:
prob = pulp.LpProblem('Transportation optimization', pulp.const.LpMinimize)

Create LP variables for reference

In [82]:
routes = [(w, h) for w in warehouses for h in houses]
_route_variables = pulp.LpVariable.dicts('route', (warehouses, houses), lowBound = 0, upBound = None, cat = pulp.LpInteger)

In [83]:
print('Tuple representation of routes')
print(routes)

print('\nPuLP variables for reference')
_route_variables

Tuple representation of routes
[('A', '1'), ('A', '2'), ('A', '3'), ('A', '4'), ('A', '5'), ('B', '1'), ('B', '2'), ('B', '3'), ('B', '4'), ('B', '5')]

PuLP variables for reference


{'A': {'1': route_A_1,
  '2': route_A_2,
  '3': route_A_3,
  '4': route_A_4,
  '5': route_A_5},
 'B': {'1': route_B_1,
  '2': route_B_2,
  '3': route_B_3,
  '4': route_B_4,
  '5': route_B_5}}

Set objective function and constraints

In [84]:
# objective function 
prob += pulp.lpSum([_route_variables[w][h] * costs[w][h] for (w, h) in routes])

#supply
for w in warehouses:
    prob += pulp.lpSum([_route_variables[w][h] for h in houses]) <= supply[w], 'supply in ' + w

#demand
for h in houses:
    prob += pulp.lpSum([_route_variables[w][h] for w in warehouses]) >= demand[h], 'demand in ' + h

Solve and check

In [85]:
status = prob.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [86]:
for variable in prob.variables():
    print("{} = {}".format(variable.name, variable.varValue))

route_A_1 = 300.0
route_A_2 = 0.0
route_A_3 = 0.0
route_A_4 = 0.0
route_A_5 = 700.0
route_B_1 = 200.0
route_B_2 = 900.0
route_B_3 = 1800.0
route_B_4 = 200.0
route_B_5 = 0.0


In [88]:
OPTIMUM = pulp.value(prob.objective)
print('\nOptimal cost of transportation ($):', OPTIMUM)


Optimal cost of transportation ($): 8600.0
